In [2]:
## Install required packages

!pip install torch diffusers transformers fastapi uvicorn nest_asyncio

  Using cached fastapi-0.115.2-py3-none-any.whl.metadata (27 kB)
  Using cached uvicorn-0.32.0-py3-none-any.whl.metadata (6.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.7/94.7 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.3/73.3 kB 7.4 MB/s eta 0:00:00


In [3]:
## Import necessary libraries

import torch
from diffusers import StableDiffusionPipeline
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
import nest_asyncio
import  uvicorn
import base64
from io import BytesIO
from PIL import Image

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [4]:
from huggingface_hub import login
login()

In [16]:
## Initialize Stable Diffusion
model_id = "stabilityai/stable-diffusion-2-1"
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16)
pipe = pipe.to("cuda")

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-7' coro=<Server.serve() done, defined at /usr/local/lib/python3.10/dist-packages/uvicorn/server.py:67> exception=KeyboardInterrupt()>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/uvicorn/main.py", line 579, in run
    server.run()
  File "/usr/local/lib/python3.10/dist-packages/uvicorn/server.py", line 65, in run
    return asyncio.run(self.serve(sockets=sockets))
  File "/usr/local/lib/python3.10/dist-packages/nest_asyncio.py", line 30, in run
    return loop.run_until_complete(task)
  File "/usr/local/lib/python3.10/dist-packages/nest_asyncio.py", line 92, in run_until_complete
    self._run_once()
  File "/usr/local/lib/python3.10/dist-packages/nest_asyncio.py", line 133, in _run_once
    handle._run()
  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "/usr/lib/python3.10/asyncio/tasks.py", l

In [17]:
## Create FastAPI app and endpoints
app = FastAPI()

class TextPrompt(BaseModel):
    text: str

@app.post("/generate")
async def generate_image(prompt: TextPrompt):
    try:
        # Generate image from text
        image = pipe(prompt.text).images[0]

        # Convert PIL image to base64
        buffered = BytesIO()
        image.save(buffered, format="PNG")
        img_str = base64.b64encode(buffered.getvalue()).decode()

        return {"image": img_str}
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

In [18]:
## Setup ngrok for public URL
!pip install pyngrok
from pyngrok import ngrok

link : https://dashboard.ngrok.com/get-started/your-authtoken

In [ ]:
# Setup ngrok
print("Setting up ngrok...")  # Debug info
ngrok.set_auth_token("xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx")  # Replace with your ngrok token
public_url = ngrok.connect(8000)
print('Public URL:', public_url.public_url)

# Start the server
print("Starting server...")  # Debug info
nest_asyncio.apply()
uvicorn.run(app, port=8000)


Setting up ngrok...
Public URL: https://a620-35-227-171-239.ngrok-free.app
Starting server...


INFO:     Started server process [388]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     2405:201:6816:305c:b128:bb6c:af8e:e918:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     2405:201:6816:305c:b128:bb6c:af8e:e918:0 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     2405:201:6816:305c:b128:bb6c:af8e:e918:0 - "GET / HTTP/1.1" 404 Not Found
INFO:     2405:201:6816:305c:b128:bb6c:af8e:e918:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     2405:201:6816:305c:b128:bb6c:af8e:e918:0 - "GET /openapi.json HTTP/1.1" 200 OK


  0%|          | 0/50 [00:00<?, ?it/s]

INFO:     2405:201:6816:305c:b128:bb6c:af8e:e918:0 - "POST /generate HTTP/1.1" 200 OK


...

INFO:     34.211.200.85:0 - "POST /generate HTTP/1.1" 200 OK


  0%|          | 0/50 [00:00<?, ?it/s]

INFO:     34.211.200.85:0 - "POST /generate HTTP/1.1" 200 OK


  0%|          | 0/50 [00:00<?, ?it/s]

INFO:     34.211.200.85:0 - "POST /generate HTTP/1.1" 200 OK
